# Main

In [1]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('preprocess.csv')

data

,name,category
0,아동영어보육과,0
1,아동영어보육학과,0
2,유아특수보육전공,1
3,특수보육계열,1
4,컴퓨터보육과,2
...,...,...
27112,스마트이동체융합시스템공학과,3
27113,학과간협동과정항만물류시스템학과,3
27114,동북아물류시스템학과,3
27115,항만물류시스템,3


In [3]:
def category_str(data, classification=['인문', '사회', '교육', '공학', '자연', '의약', '예체능']):
    copy_data = deepcopy(data)
    for i in range(7):
        copy_data.loc[ copy_data['category']==i , 'category' ] = classification[i]
    return copy_data

In [4]:
def preprocess_name(text):
    result = ''
    
    # 빈칸제거
    result = text.strip()
    
    # 한글만 추출
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')  # 한글과 띄어쓰기를 제외한 모든 글자
    result = hangul.sub('', result)
    
    # 1차
    remove_words = ['학과', '전공', '학부', '대학', '계열', '단과대학없음', '프로그램', '과']
    result = result.strip().replace(' ', '').strip()
    
    if result in remove_words:
        return '0'
    else:
        return result

In [5]:
data1 = category_str(data)

data1

,name,category
0,아동영어보육과,인문
1,아동영어보육학과,인문
2,유아특수보육전공,사회
3,특수보육계열,사회
4,컴퓨터보육과,교육
...,...,...
27112,스마트이동체융합시스템공학과,공학
27113,학과간협동과정항만물류시스템학과,공학
27114,동북아물류시스템학과,공학
27115,항만물류시스템,공학


In [6]:
# data2 = category_str(data, ['인문/사회', '인문/사회', '교육', '공학', '자연', '의약', '예체능'])

# data2

In [7]:
x = data1['name'].to_list()

x

['아동영어보육과',
 '아동영어보육학과',
 '유아특수보육전공',
 '특수보육계열',
 '컴퓨터보육과',
 '가정보육과',
 '보육과',
 '보육상담전공',
 '보육학과',
 '사회복지보육과',
 '생활보육과',
 '아동미술보육전공',
 '아동보육과',
 '아동복지보육과',
 '영유아보육과',
 '유아교육과',
 '유아교육전공',
 '유아교육학과',
 '유아미술음악전공',
 '아동미술보육과',
 '아동미술복지보육과',
 '아동보육학습지도과',
 '아동생활보육과',
 '아동컴퓨터보육과',
 '아동보육전공',
 '방과후아동보육과',
 '아동보육복지과',
 '유아특수보육과',
 '아동미술보육계열',
 '아동보육계열',
 '유아보육과',
 '유아보육미술과',
 '사회복지보육전공',
 '특수보육과',
 '아동문학보육과',
 '아동미술복지보육전공',
 '유아교육과년제',
 '영유아보육복지과',
 '아동복지미술보육과',
 '유아교육과인문',
 '미술보육과',
 '아동보육복지학과',
 '영유아보육전공',
 '아동미디어미술보육과',
 '아동표현미술보육과',
 '유아교육학과년제',
 '영재미술보육전공',
 '보육복지과',
 '아동보육학과',
 '영유아보육학부',
 '어린이영어보육과',
 '아동보육복지전공',
 '사회복지과아동보육복지전공',
 '유아특수보육과년제',
 '아동보육복지상담과',
 '아동보육과인문사회',
 '아동미술보육과년제',
 '유아보육문화계열',
 '영유아보육학과',
 '아동복지보육과년제',
 '유아보육계열',
 '유아안전보육과',
 '아동건강보육과',
 '아동보육과년제',
 '아동문화계열',
 '아동상담보육과',
 '유아특수보육학과',
 '아동컴퓨터보육전공',
 '아동조형미술보육과',
 '아동미디어미술보육전공',
 '아동표현미술보육전공',
 '아동조형미술보육전공',
 '유아특수치료교육',
 '유아특수치료교육과',
 '유아특수재활과',
 '장애유아보육과',
 '특수아동재활과',
 '유아특수언어재활과',
 '유아특수재활복지과',
 '특수아동언어재활과',
 '유아특

In [8]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embeddings = model.encode(x)

embeddings.shape

(27117, 768)

In [9]:
scaler = StandardScaler()
scale_embeddings = scaler.fit_transform(embeddings)

scale_embeddings.shape

(27117, 768)

In [10]:
def pca_decide(n, data=scale_embeddings):
    pca = PCA(n_components=n)  # 주성분을 몇개로 할지 결정
    test = pca.fit_transform(data)
    df = pd.DataFrame(data=test, columns=[f"pca{num+1}" for num in range(n)])
    
    var = pca.explained_variance_
    ratio = pca.explained_variance_ratio_

    results = pd.DataFrame({'설명가능한 분산 비율(고윳값)':var, '기여율':ratio},
                         index = np.array([f"pca{num+1}" for num in range(n)]))
    results['누적기여율'] = results['기여율'].cumsum()
    
    if results['누적기여율'][-1] >= 0.8:
        return results
    

def find_dimension():
    for n in range(3, 100):
        result = pca_decide(n)
        try:
            if result['설명가능한 분산 비율(고윳값)'][-1] >= 0.7:
                print(result)
                return n
        except:
            pass

In [11]:
dimension = find_dimension()

print(f"{dimension} 차원")

       설명가능한 분산 비율(고윳값)       기여율     누적기여율
pca1          42.979954  0.055962  0.055962
pca2          36.224880  0.047166  0.103128
pca3          33.651062  0.043815  0.146943
pca4          26.627037  0.034669  0.181612
pca5          23.817720  0.031012  0.212624
...                 ...       ...       ...
pca74          2.195573  0.002859  0.791218
pca75          2.120348  0.002761  0.793978
pca76          2.068621  0.002693  0.796672
pca77          1.986994  0.002587  0.799259
pca78          1.959176  0.002551  0.801810

[78 rows x 3 columns]
78 차원


In [12]:
pca = PCA(n_components=dimension) # 주성분을 몇개로 할지 결정
test = pca.fit_transform(scale_embeddings)
X = pd.DataFrame(data=test, columns=[f"pca{num+1}" for num in range(dimension)])

X

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca69,pca70,pca71,pca72,pca73,pca74,pca75,pca76,pca77,pca78
0,-10.551479,-0.762731,-0.160632,7.201928,2.393260,2.654570,6.030083,1.441224,8.255486,-1.908378,...,0.221720,-2.409103,-1.673186,-0.756653,0.614827,-0.819092,1.381082,-0.623759,-2.424866,-0.077226
1,-8.657638,-1.748011,1.240447,6.341136,-2.442706,3.103917,3.576908,-2.316793,4.974167,-1.695292,...,-0.059067,-2.071149,-1.889170,-1.706588,0.083291,-0.808909,0.153521,-0.733416,-1.203504,0.874869
2,-4.485749,-2.786324,-0.961337,9.138257,-3.725617,3.920282,-3.522173,1.608502,6.323431,-1.267410,...,2.392258,-0.910562,2.573691,-3.021610,1.085898,0.193341,1.487857,-0.655421,1.408151,0.029030
3,-5.706581,-1.581073,-3.901394,8.444678,0.884133,1.012768,-1.870622,3.692834,4.373931,-1.917200,...,0.606083,4.369171,2.752354,-0.027132,-0.060561,0.765681,1.126124,-0.367987,2.333750,-0.218369
4,-2.114196,-1.476694,5.808391,10.699753,5.400708,4.259670,4.669149,2.216517,6.183794,-2.881443,...,-0.208086,-0.679222,-0.859166,-0.785574,2.163347,-0.070720,1.285268,-0.141363,-0.961059,-1.032000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27112,9.792577,-3.351534,6.869893,0.304703,3.303103,-0.960789,-2.588664,4.521887,1.489631,7.109571,...,2.014209,0.682003,0.580205,-0.838247,-1.272092,-0.554641,-0.793883,-0.158515,-0.305686,0.606480
27113,4.715868,-4.266416,0.707204,-6.709468,1.166968,-0.377802,-6.187320,1.759468,6.464289,-0.776180,...,-0.416352,0.269576,1.864515,-0.500503,-0.443329,1.177003,1.094961,-0.079108,-0.691723,0.558612
27114,-0.030055,-5.204022,1.515093,-13.278954,3.867899,-2.989593,-0.760998,1.778659,4.786046,0.246161,...,-0.108531,-0.530416,3.450575,-3.023554,-0.146939,-0.403648,1.253774,-0.737982,1.523596,-0.361675
27115,-0.031045,-2.405831,0.565071,-9.701746,11.256278,-3.153958,2.138318,5.574585,5.876939,0.915296,...,-1.702584,-0.995358,1.506295,-0.761187,-0.050173,-1.507259,2.368310,-1.532613,0.451949,3.027912


In [13]:
y = data1['category']

y

0        인문
1        인문
2        사회
3        사회
4        교육
         ..
27112    공학
27113    공학
27114    공학
27115    공학
27116    공학
Name: category, Length: 27117, dtype: object

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(21693, 78)
(5424, 78)
(21693,)
(5424,)


In [15]:
rfm = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf=3)
rfm.fit(x_train.values, y_train.values)

RandomForestClassifier(max_depth=20, min_samples_leaf=3)

In [16]:
pred = rfm.predict(x_test.values)

print(y_test.values, '\n')
print(pred)

['공학' '자연' '교육' ... '공학' '예체능' '사회'] 

['공학' '자연' '교육' ... '공학' '예체능' '사회']


In [17]:
acc = accuracy_score(y_test, pred)

print('Acc: {:.2f}%'.format(acc*100))

Acc: 83.00%


# find fail

In [18]:
y_test

24037     공학
17785     자연
20375     교육
13784     자연
9070     예체능
        ... 
1686      인문
14349     공학
14662     공학
7894     예체능
11852     사회
Name: category, Length: 5424, dtype: object

In [19]:
pred

array(['공학', '자연', '교육', ..., '공학', '예체능', '사회'], dtype=object)

In [20]:
y_test.index

Int64Index([24037, 17785, 20375, 13784,  9070, 12943,  6141, 26458, 24740,
            10734,
            ...
            25790, 24949,   395,  7318, 20697,  1686, 14349, 14662,  7894,
            11852],
           dtype='int64', length=5424)

In [21]:
index_list, leng = [], len(y_test)
for yt, pr, i in zip(y_test, pred, [l for l in range(leng)]):
    if yt != pr:
        index_list.append(i)

In [22]:
# 틀린 과목 리스트
x_name = []
for i in index_list:
    x_name.append(x[i])
    
x_name

['보육상담전공',
 '아동미술보육전공',
 '아동보육과',
 '아동보육전공',
 '유아교육과년제',
 '아동표현미술보육과',
 '유아교육학과년제',
 '영재미술보육전공',
 '아동복지보육과년제',
 '유아안전보육과',
 '아동건강보육과',
 '아동문화계열',
 '아동상담보육과',
 '아동예능지도과',
 '아동체육무용과',
 '아동무용지도자과',
 '아동미술과',
 '아동미술전공',
 '아동조형미술과',
 '아동미술지도전공',
 '아동미술지도과',
 '환경교육과',
 '아동평생교육학부',
 '생활문화과',
 '디자인학전공',
 '디자인과',
 '디지털미디어과',
 '문예영상창작과',
 '방송극작',
 '방송기술과',
 '방송매체과',
 '방송보도제작계열',
 '방송영상사진과',
 '방송제작과',
 '신문방송과',
 '인터넷방송전공',
 '아동미디어과',
 '인터넷신문방송과',
 '방송연예제작학부',
 '미디어커뮤니케이션과',
 '방송연예음악전공',
 '영상시나리오창작과',
 '방송미디어학과',
 '방송제작전공',
 '방송영상디지털계열',
 '첨단방송영상학과',
 '미디어출판학과',
 '컴퓨터멀티미디어전공',
 '게임소프트웨어전공',
 '스마트폰미디어학과',
 '디지털방송콘텐츠학과',
 '게임제작계열',
 '인터넷게임전공',
 '컴퓨터정보게임계열',
 '멀티게임영상과',
 '영상게임콘텐츠과',
 '신산업창조학부전공',
 '멀티미디어통신학과',
 '출판미디어과',
 '시각디자인계열',
 '시각디자인과',
 '시각정보디자인과',
 '캐릭터시각디자인전공',
 '디자인학부산업디자인과시각디자인전공',
 '시각디자인과년제',
 '시각디자인과시각정보디자인전공',
 '광고멀티미디어디자인전공',
 '그래픽디자인',
 '디지털미디어디자인학과',
 '디지털미디어디자인학부',
 '출판디자인전공',
 '캐릭터디자인전공',
 '광고영상디자인전공',
 '디지털디자인전공',
 '캐릭터디자인과',
 '컴퓨터광고디자인전공',
 '산업디자인계열컴퓨터그래픽디자인전공',
 '컴퓨터영상디자인과',
 

# Seogang Test

In [23]:
seogang_class = [
    '인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', 
    '사회', '사회', '사회', '사회', '사회', 
    '예체능', '예체능',  '예체능', '예체능', '예체능', '예체능', 
    '자연', '자연', '자연', '자연', 
    '공학', '공학', '공학', '공학', '공학', '공학'
]

seogang_major = [
    '국어국문학', '사학', '철학', '종교학', '영문학부', '유럽문화', '독일문화', '프랑스문화', '중국문화', '일본문화', 
    '사회학', '정치외교학', '심리학', '경제학', '경영학', 
    '국제한국학', '아트&테크놀로지', '신문방송학', '미디어&엔터테인먼트', '글로벌 한국학', '커뮤니케이션학', 
    '수학', '물리학', '화학', '생명과학', 
    '전자공학', '화공생명공학', '컴퓨터공학', '기계공학', '인공지능학', '시스템반도체공학'
]

In [24]:
majors = []
for word in seogang_major:
    majors.append(preprocess_name(word))
    
majors

['국어국문학',
 '사학',
 '철학',
 '종교학',
 '영문학부',
 '유럽문화',
 '독일문화',
 '프랑스문화',
 '중국문화',
 '일본문화',
 '사회학',
 '정치외교학',
 '심리학',
 '경제학',
 '경영학',
 '국제한국학',
 '아트테크놀로지',
 '신문방송학',
 '미디어엔터테인먼트',
 '글로벌한국학',
 '커뮤니케이션학',
 '수학',
 '물리학',
 '화학',
 '생명과학',
 '전자공학',
 '화공생명공학',
 '컴퓨터공학',
 '기계공학',
 '인공지능학',
 '시스템반도체공학']

In [25]:
emb = model.encode(majors)

scale_emb = scaler.transform(emb)

scale_emb.shape

(31, 768)

In [26]:
tes = pca.transform(scale_emb)

tes.shape

(31, 78)

In [27]:
pre = rfm.predict(tes)

print(seogang_class, '\n')
print(pre)

['인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', '인문', '사회', '사회', '사회', '사회', '사회', '예체능', '예체능', '예체능', '예체능', '예체능', '예체능', '자연', '자연', '자연', '자연', '공학', '공학', '공학', '공학', '공학', '공학'] 

['인문' '인문' '인문' '인문' '인문' '인문' '인문' '인문' '인문' '인문' '인문' '사회' '인문' '사회'
 '사회' '인문' '예체능' '사회' '사회' '인문' '사회' '교육' '자연' '인문' '자연' '공학' '공학' '공학'
 '공학' '공학' '공학']


In [28]:
Acc = accuracy_score(seogang_class, pre)

print('Acc: {:.2f}%'.format(Acc*100))

Acc: 70.97%
